In [ ]:
#@title Automatically find threshold and save histogram with corresponding CA simulation

import numpy as np
import matplotlib.pyplot as plt
import sys
from collections import Counter
np.set_printoptions(threshold=sys.maxsize)
import copy
thresholds = []
def generate_rule(rule_number):
    rule = np.zeros(8, dtype=int)
    binary = np.binary_repr(rule_number, width=8)
    for i in range(8):
        rule[i] = int(binary[i])
    return rule

def apply_rule(rule, neighborhood):
    decimal = np.sum(neighborhood * np.array([1, 2, 4]))
    return rule[7 - decimal]

def simulate(rule_number, num_iterations, num_cells, initial_percentage):
    np.random.seed(1)
    rule = generate_rule(rule_number)
    initial_state = np.zeros(num_cells, dtype=int)
    num_initial_cells = int(num_cells * initial_percentage / 100)
    initial_indices = np.random.choice(num_cells, num_initial_cells, replace=False)
    initial_state[initial_indices] = 1

    state = np.zeros((num_iterations, num_cells), dtype=int)
    state[0] = initial_state

    for i in range(1, num_iterations):
        for j in range(num_cells):
            neighborhood = state[i - 1, (j - 1) % num_cells], state[i - 1, j], state[i - 1, (j + 1) % num_cells]
            state[i, j] = apply_rule(rule, neighborhood)

    return state


def blockshaped(arr, n):
    assert len(arr) % n == 0, f"Length of array is not evenly divisible by {n}"
    return arr.reshape(-1, n)

def process_FHCG(state, rule_number, initial_percentage):
    time_steps, height = state.shape
    supercells_list = []
    new_grid_supercells = []
    block_size = 2

    for array in state:
        blocked_chunks = blockshaped(array, block_size)
        for each_chunk in blocked_chunks:
            supercells_list.append(list(each_chunk.flatten()))

    test_array = np.array(supercells_list)

    for i in range(len(supercells_list)):
        count = 0
        for j in range(len(supercells_list)):
            if supercells_list[i] == supercells_list[j]:
                count = count + 1
        new_grid_supercells.append(count / len(supercells_list))

    a = supercells_list
    total = len(a)
    freq = Counter(map(tuple, a))
    prob = {k: v / total for k, v in freq.items()}
    sorted_prob = dict(sorted(prob.items(), key=lambda x: x[0]))
    list_probs = list(sorted_prob.values())


    four_simulations_diff_threhsolds = []
    thresholds = [a+0.01 for a in list_probs]
    # print(thresholds)
    for each_threshold in thresholds:
      fig, ax = plt.subplots()
      ax.bar(range(len(sorted_prob)), list(sorted_prob.values()), color='steelblue', edgecolor='black')
      ax.set_xticks(range(len(sorted_prob)))
      ax.set_xticklabels([int(''.join(map(str, k)), 2) for k in sorted_prob.keys()], rotation=90)
      ax.set_xlabel('Decimal Values of the binary values in block')
      ax.set_ylabel('Probability of occurrence')
      ax.set_title('Probability of occurrence of supercells vs (Decimal Values)')
      ax.set_yticks([i / 20 for i in range(21)])
      ax.set_yticklabels([f'{i / 20:.2f}' for i in range(21)])
      ax.hlines(each_threshold, xmin=range(len(sorted_prob))[0], xmax=range(len(sorted_prob))[-1], colors='r', linestyles='dashed')
      ax.set_ylim(0, 0.9)
      ax.spines['top'].set_visible(False)
      ax.spines['right'].set_visible(False)
      ax.grid(color='lightgray')

      ax.annotate(f'Threshold: {each_threshold}', xy=(range(len(sorted_prob))[-1], each_threshold), xytext=(-45, 5), textcoords='offset points', color='r')

      plt.savefig("histogram_{}_init_{}_threshold_{}.svg".format(rule_number, initial_percentage, each_threshold),format='svg')
      plt.close()
      temp_new_grid_supercells = copy.deepcopy(new_grid_supercells)
      for k in range(len(new_grid_supercells)):
          if new_grid_supercells[k] <= each_threshold:
              temp_new_grid_supercells[k] = 1
          else:
              temp_new_grid_supercells[k] = 0

      modified_board = np.array(temp_new_grid_supercells)
      shape = int(height / block_size)
      modified_automaton = modified_board.reshape(time_steps, shape)
      four_simulations_diff_threhsolds.append(modified_automaton)
    modified_automatons = four_simulations_diff_threhsolds
    return modified_automatons,thresholds





def visualize(rule_number, num_iterations, num_cells, initial_percentage):
    state = simulate(rule_number, num_iterations, num_cells, initial_percentage)
    states_cg,thresholds = process_FHCG(state,rule_number,initial_percentage)
    for i in range(len(thresholds)):
      fig, ax = plt.subplots()
      ax.imshow(state, cmap='Greys', interpolation='nearest')
      plt.savefig(f'rule_{rule_number}_init_{initial_percentage}_threshold{thresholds[i]}.svg',format='svg')
      plt.close()
      plt.imshow(states_cg[i], cmap='Greys', interpolation='nearest')
      plt.savefig(f'cg_rule_{rule_number}_init_{initial_percentage}_threshold{thresholds[i]}.svg',format='svg')
      plt.close()
    return thresholds



# Parameters
num_cells = 100
num_iterations = 100


# use this for main run
initial_percentages = [10]  # Add more percentages if desired



# Simulate and visualize all possible 1D cellular automata rules with random initialization percentages
for rule_number in range(60,61): # left value inclusive and right value exclusive
    for initial_percentage in initial_percentages:
      thresholds = visualize(rule_number, num_iterations, num_cells, initial_percentage)


In [ ]:
# !zip -q exsport.zip *.svg